In [1]:
# Open CV에서 얼굴 정렬하는 방법
'''
1. 이미지에서 얼굴과 눈 감지
2. 감지된 눈의 중심 계산
3. 두 눈 중앙 사이에 선을 긋기
4. 두 눈 사이의 수평선 그리기
5. 삼각형의 세 변의 길이 계산 및 각도 계산
6. 계산된 각도로 이미지 회전
7. 이미지 크기 조정
'''

'\n1. 이미지에서 얼굴과 눈 감지\n2. 감지된 눈의 중심 계산\n3. 두 눈 중앙 사이에 선을 긋기\n4. 두 눈 사이의 수평선 그리기\n5. 삼각형의 세 변의 길이 계산\n6. 각도 계산\n7. 계산된 각도로 이미지 회전\n8. 이미지 크기 조정\n'

In [24]:
## 1. 이미지에서 얼굴과 눈 감지
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("haarcascade_eye.xml")

## 1-1. 얼굴 이미지 데이터 읽기
img = cv2.imread('face.png')
cv2.imshow("image show", img)
cv2.waitKey(0)

-1

In [25]:
## 1-2. 얼굴 이미지 바운딩 박스
# gray스케일로 변환
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 바운딩 박스 좌표 획득
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

# 박스 그리기
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 3)
cv2.imshow("face", img)
cv2.waitKey(0)

-1

In [26]:
## 1-3. 눈 감지
# 관심영역 설정
roi_gray = gray[y:(y+h), x:(x+w)]
roi_color = img[y:(y+h), x:(x+w)]
 
# 눈 바운딩 박스 좌표 설정
eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 4) # gray스케일 이미지 필요
index = 0

# 오른쪽, 왼쪽 눈 박스 그리기
for (ex, ey, ew, eh) in eyes:
    if index ==0:
        eye_1 = (ex, ey, ew, eh)
    elif index == 1:
        eye_2 = (ex, ey, ew, eh)

# 눈 박스 그리기
    cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0,0,255), 3)
    index += 1

cv2.imshow("face", img)
cv2.waitKey(0)


-1

In [23]:
## 2. 감지된 눈의 중심 계산
# 왼쪽 오른쪽 눈 기준 설정
if eye_1[0] < eye_2[0]:
    left_eye = eye_1
    right_eye = eye_2
else:
    left_eye = eye_2
    right_eye = eye_1

# 눈 박스 중앙점 계산
left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
left_eye_x = left_eye_center[0]
left_eye_y = left_eye_center[1]

right_eye_center = (int(right_eye[0] + (right_eye[2] / 2)), int(right_eye[1] + (right_eye[3] / 2)))
right_eye_x = right_eye_center[0]
right_eye_y = right_eye_center[1]

## 3. 두 눈 중앙 사이에 선 긋기
cv2.circle(roi_color, left_eye_center, 5, (255,0,0), -1)
cv2.circle(roi_color, right_eye_center, 5, (255,0,0), -1)
cv2.line(roi_color, right_eye_center, left_eye_center, (0,200,200), 3)

cv2.imshow("face", img)
cv2.waitKey(0)

-1

In [8]:
## 4. 두 눈 사이의 수평선 그리기
if left_eye_y > right_eye_y:
    A = (right_eye_x, left_eye_y)
    # Integer -1 indicates that the image will rotate in the clockwise direction 
    direction = -1 # 시계 방향
else:
    A = (left_eye_x, right_eye_y)
    direction = 1 # 반시계 방향

cv2.circle(roi_color, A, 5, (255,0,0), -1)

cv2.line(roi_color, right_eye_center, left_eye_center, (0,200,200),3)
cv2.line(roi_color, left_eye_center, A, (0,200,200),3)
cv2.line(roi_color, right_eye_center, A, (0,200,200),3)

cv2.imshow("face", img)
cv2.waitKey(0)


-1

In [10]:
## 5. 삼각형의 세변의 길이 계산 및 각도 계산
# np.arctan 함수는 라디안 단위로 각도를 반환 한다는 점에 유의.
# 결과를 각도로 변환하려면
# 각도 \(\theta(세타) \ )에 180을 곱한 다음 \(\pi \)로 나누어야 한다.
delta_x = right_eye_x - left_eye_x
delta_y = right_eye_y - left_eye_y
angle = np.arctan(delta_y / delta_x)
angle = (angle * 180) / np.pi
angle

-21.80140948635181

In [13]:
## 6. 계산된 각도로 이미지 회전
# Width and height of the image
h, w = img.shape[:2] # img.shape = (300, 332, 3)

# Calculating a center point of the image
# Integer division "//"" ensures that we receive whole number
center = (w // 2, h// 2)

# Defining a matrix M and calling
# cv2.getRotationMatrix2D method
M = cv2.getRotationMatrix2D(center, (angle), 1.0)

# Applying the roration to our image using the
# cv2.warpAffine method
rotated = cv2.warpAffine(img, M, (w, h))
cv2.imshow("face", rotated)
cv2.waitKey(0)

-1

In [ ]:
## 7. 이미지 크기 조정
# 비율에 따라 이미지 조정